# ReasoningAgnet (Tree of Thoughts with Beam Search)

## Introduction

This repository demonstrates how to implement a Tree of Thoughts (ToT) reasoning agent using beam search in under 50 lines of core code (excluding auxiliary components like visualization and logging). The `ReasoningAgent` leverages large language models (LLMs) such as GPT-4 or Llama to perform complex reasoning tasks by exploring multiple thought trajectories.

In the provided example, we showcase how the `ReasoningAgent`, even when using a smaller model (e.g., `gpt-4o-mini`), can effectively generate and evaluate multiple reasoning paths to solve intricate problems.

## What is the Tree of Thoughts and Beam Search?

The **Tree of Thoughts (ToT)** is a reasoning framework where an agent considers multiple possible reasoning steps (thoughts) and organizes them into a tree structure. **Beam search** is an optimization algorithm that explores a subset of the tree by keeping the top `k` options at each level (the beam size). Combining ToT with beam search allows the agent to efficiently navigate through the most promising thought trajectories.

Use **Beam Size = 1** for a special case of O1-style reasoning, which is similar to CoT with multiple prompts. 


## Key Features

- **Enhanced Problem-Solving**: Implements the Tree of Thoughts framework to improve reasoning capabilities.
- **Beam Search Optimization**: Utilizes beam search to efficiently explore and evaluate multiple thought paths.
- **Multi-Agent Collaboration**: Includes a thinker agent to generate possible next steps and a grader agent to evaluate them.
- **Visualization Support**: Provides visualization of the thought tree using Graphviz for better analysis.
- **Customizable Parameters**: Allows configuration of maximum depth, beam size, and answer selection approach.

In [1]:
import os

api_key = os.environ.get("OPENAI_API_KEY")

config_list = [
    {
        "model": "gpt-4o-mini",
        "api_key": api_key
    }
]
verbose = False

## Simple Chain-of-Thought O1-style 

In [2]:
from autogen import AssistantAgent, UserProxyAgent
from autogen.agentchat.contrib.reasoning_agent import ReasoningAgent, visualize_tree

thought_agent = ReasoningAgent(
    name="thought_agent",
    llm_config={"config_list": config_list},
    verbose=verbose,
    beam_size=1,  # using beam size 1 for special case of O1-style reasoning.
    max_depth=3,
)
user_proxy = UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    code_execution_config={"use_docker": False},
    max_consecutive_auto_reply=10,
)

In [3]:
question = "What is the expected maximum value when rolling a 6-sided dice three times?"

def last_meaningful_msg(sender, recipient, summary_args):
    import warnings

    if sender == recipient:
        return "TERMINATE"

    summary = ""
    chat_messages = recipient.chat_messages[sender]

    for msg in reversed(chat_messages):
        try:
            content = msg["content"]
            if isinstance(content, str):
                summary = content.replace("TERMINATE", "")
            elif isinstance(content, list):
                # Remove the `TERMINATE` word in the content list.
                summary = "\n".join(
                    x["text"].replace("TERMINATE", "") for x in content if isinstance(x, dict) and "text" in x
                )
            if summary.strip().rstrip():
                return summary
        except (IndexError, AttributeError) as e:
            warnings.warn(f"Cannot extract summary using last_msg: {e}. Using an empty str as summary.", UserWarning)
    return summary

In [4]:
ans = user_proxy.initiate_chat(thought_agent, message=question, summary_method=last_meaningful_msg)

user_proxy (to thought_agent):

What is the expected maximum value when rolling a 6-sided dice three times?

--------------------------------------------------------------------------------
thought_agent (to tot_thinker):

# Question: What is the expected maximum value when rolling a 6-sided dice three times?
---
What are the possible next steps?

--------------------------------------------------------------------------------
tot_thinker (to thought_agent):

**Reflection**  
The previous steps do not seem to indicate any errors, but rather a lack of clarity on how to approach calculating the expected maximum value when rolling a 6-sided dice three times. The user might need more structured insight into both the probability theory involved and how to derive that expected maximum. The foundational concepts related to rolls and their outcomes have not been explicitly laid out.

**Possible Options:**  
Option 1: Calculate the possible outcomes for each of the three dice rolls and determin

In [5]:
print(ans.summary)

The expected maximum value when rolling a 6-sided die three times is approximately \( 4.9583 \). Let's break down the calculation and methodology step-by-step to enhance clarity.

### Step-by-Step Breakdown

1. **Understanding the Expectation Formula**:
    - The expected maximum \( E[\max(X_1, X_2, X_3)] \), where \( X_1, X_2, \) and \( X_3 \) are the outcomes of three separate rolls of a die, can be mathematically expressed using probabilities.

2. **Calculating the Probabilities**:
    - For any outcome \( x \) (from 1 to 6), we determine the probability that the maximum of the three rolls is exactly \( x \). 
    - The probability that the maximum of the three rolls is less than or equal to \( x \) is given by:
      \[
      P(\max(X_1, X_2, X_3) \leq x) = \left(\frac{x}{6}\right)^3
      \]
    - The probability that the maximum equals \( x \) is the difference between the probabilities of the maximum being at most \( x \) and at most \( x-1 \):
      \[
      P(\max(X_1, X_2, X_

## Larger Beam Size Search in Tree of Thought

In [6]:
thought_agent = ReasoningAgent(
    name="thought_agent",
    llm_config={"config_list": config_list},
    verbose=verbose,
    beam_size=3,
    max_depth=3,
)
user_proxy = UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    code_execution_config={"use_docker": False},
    max_consecutive_auto_reply=10,
)

In [7]:
ans = user_proxy.initiate_chat(thought_agent, message="Design a mixed integer linear program for a coffee roasting supply chain", summary_method=last_meaningful_msg)

user_proxy (to thought_agent):

Design a mixed integer linear program for a coffee roasting supply chain

--------------------------------------------------------------------------------
thought_agent (to tot_thinker):

# Question: Design a mixed integer linear program for a coffee roasting supply chain
---
What are the possible next steps?

--------------------------------------------------------------------------------
tot_thinker (to thought_agent):

**Reflection**  
The previous steps do not seem to include any specific actions taken toward the design of the mixed integer linear program (MILP) for a coffee roasting supply chain. Given that the user's question is quite broad, it appears there may have been an opportunity to clarify requirements or constraints needed for the programmed solution. However, there is no immediate error to correct since we are starting fresh at this point.

**Possible Options:**  
Option 1: Define the key components of the coffee roasting supply chain, su

In [8]:
print(ans.summary)

To design a mixed integer linear program (MILP) for a coffee roasting supply chain, I will follow a structured approach leveraging elements from the proposed possibilities. Here's the plan:

1. **Identify and Formulate the Objective Function**: The objective function will focus on minimizing costs within the coffee roasting supply chain. This includes costs associated with raw materials (coffee beans), roasting costs, labor costs, transportation, and inventory holding costs.

2. **Define Decision Variables**: These will include:
   - The quantity of different types of coffee beans to purchase.
   - The amount of coffee to roast at each time period.
   - The number of workers scheduled or hours worked.
   - Inventory levels of roasted coffee.

3. **Establish Constraints**: We need constraints for:
   - Capacity constraints of the roasting facilities.
   - Demand constraints to ensure that customer demand is met for each product type.
   - Lead time constraints to manage the timing of ro

In [9]:
!pip install graphviz

In [14]:
visualize_tree(thought_agent._root)

![](tree_of_thoughts.png)

### ReasoningAgent with Nested Chats

In [11]:
writer = AssistantAgent(
    name="Writer",
    llm_config={"config_list": config_list},
    system_message="""
    You are a professional writer, known for your insightful and engaging articles.
    You transform complex concepts into compelling narratives.
    You should improve the quality of the content based on the feedback from the user.
    """,
)
thought_agent_for_writer = ReasoningAgent(
    name="thought_agent",
    llm_config={"config_list": config_list},
    verbose=verbose,
    beam_size=1,
    max_depth=3,
)

def reflection_message(recipient, messages, sender, config):
    print("Reflecting...", "yellow")
    return f"Reflect, Reason and provide critique on the following writing. \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}"


In [12]:

user_proxy.register_nested_chats(
    [{"recipient": thought_agent_for_writer, "message": reflection_message, "summary_method": "last_msg", "max_turns": 1}],
    trigger=writer,
)

In [13]:
task = """Write a concise but engaging blogpost about Nvida."""

res = user_proxy.initiate_chat(recipient=writer, message=task, max_turns=2, summary_method="last_msg")

user_proxy (to Writer):

Write a concise but engaging blogpost about Nvida.

--------------------------------------------------------------------------------
Writer (to user_proxy):

### The Rise of NVIDIA: Powering the Future of Technology

In the world of technology, few companies have managed to redefine an industry like NVIDIA. Originally founded in 1993, NVIDIA carved its niche by revolutionizing graphics processing units (GPUs), essential for rendering stunning visuals in video games. However, as the digital landscape has evolved, so has NVIDIA, positioning itself at the forefront of groundbreaking innovations in artificial intelligence (AI), self-driving cars, and high-performance computing.

#### The Graphics Giant

At its core, NVIDIA’s strength remains in its GPUs. With the launch of the GeForce series, the company became synonymous with high-quality gaming, providing enthusiasts with unparalleled graphics and performance. But NVIDIA didn’t stop there. As gaming turned into a